In [1]:
from IPython.display import Markdown
import pandas as pd
from tulip.core.collection import TulipCollection
from tulip.plots import plot_line, switch_trace_to_secondary_axis
from tulip.data.bloomberg import BloombergClient as bb
from tulip.data.haver import HaverClient as hc
from tulip.data.gs import GSClient as gs
from tulip.analysis.country_related.analytics import summarize_gs_eco_fct
from tulip.plots import plot_lines

import pycountry

ISO_2 = "SE"
haver_code = 144
pyctry = pycountry.countries.get(alpha_2=ISO_2)

# Data
economic_tendency = hc.get_series(
    "SENVET@NORDIC"
)  # [Sweden: Economic Tendency Indicator (NSA, Mean Value=100)]
pmi = hc.get_series("SESVPTG@NORDIC")  # [Sweden: Composite PMI (SA, 50+=Expansion)]
confidence_indicator = hc.get_series(
    "SESVXJ@NORDIC"
)  #  [Sweden: Confidence Indicator: Total Industry (SA, 100=Mean)]
unemployment_rate = hc.get_series(
    "SESELUR@NORDIC"
)  # [Sweden: Unemployment Rate (SA, %)]
unemployment_2_rate = hc.get_series(
    "SESELUCR@NORDIC"
)  #   [Sweden: Registered Unemployed (EOP, SA, %)]
nairu = hc.get_series(
    "SEAVELUR@NORDIC"
)  #   [Sweden:  NIER Equilibrium Unemployment, Percent of Potential Labor Force (%)]

# High Frequency
job_starts = hc.get_series("E144TVNS@INTWKLY")  # [Sweden: Job Starts (NSA)]

Haver path setting remains unchanged.



### Sweden
### Activity Indicators
#### Economic Forecasts (Brokers)

In [2]:
gs_eco_fct = gs.get_eco_forecast(geographyId=ISO_2)
gs_summary = summarize_gs_eco_fct(gs_eco_fct)
gs_summary = gs_summary[~gs_summary.index.str.contains("ngdp")].to_frame().T
gs_summary.style.set_caption(f"Goldman {pyctry.name} Economic Forecasts").format(
    precision=2
)

metric,core_cpi,cpi_avg,current_account,rgdp_qoq,rgdp_yoy,output_gap
forecastValue,2.84,2.70,nan,1.68,1.48,nan


#### Current Activity Indicator (Goldman)

In [3]:
cai_series_soft_vs_hard = gs.get_CAI_series(
    geographyId=ISO_2,
    metricName=[
        "CAI_HEADLINE",
        "CAI_CONTRIBUTION_TYPE_HARD",
        "CAI_CONTRIBUTION_TYPE_SOFT",
    ],
    startDate="1980-01-01",
)
cai_series_soft_vs_hard = cai_series_soft_vs_hard.set_index("metricName", append=True)[
    "metricValue"
].unstack("metricName")
cai_series_soft_vs_hard.columns = ["Hard", "Soft", "Headline"]
cai_plot = plot_lines(
    cai_series_soft_vs_hard,
    show_0=True,
    title=f"<b>{pyctry.name} Current Activity Indicator</b> Updated: {pd.Timestamp.today().strftime('%Y-%m-%d')}",
    years_limit=4,
)
cai_plot

#### Economic Tendency

In [4]:
fig = economic_tendency.plot()
fig.add_hline(y=100, line_width=1, line_dash="dash", line_color="black")
fig.show()

### Growth Stats
#### Flash GDP

In [5]:
hc.get_series("SEWNGCY@NORDIC").plot(tick_suffix="%", show_0=True)

#### Dashboard GDP

In [6]:
growth_stats = {
    "Real GDP": "SESNGDP@NORDIC",
    "Fixed Investment": "SESNI@NORDIC",
    "Inventories": "SESNB@NORDIC",
    "Gov Consumption": "SESNCP@NORDIC",
    "HH Consumption": "SESNCV@NORDIC",
    "Exports": "SESNX@NORDIC",
    "Imports": "SESNM@NORDIC",
}

growth_stats_collection = []
for k, v in growth_stats.items():
    growth_stats_collection.append(hc.get_series(v))

growth_stats_collection = TulipCollection(growth_stats_collection)
growth_stats_collection["SESNM@NORDIC"].good_is = -1
growth_stats_collection.dashboard.table()


,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Sweden: Gross Domestic Product (SA, Mil.SEK)","1,622,649",2025-06-30,"1,621,531","1,118",0.09,"17,386","42,671",2025-11-12 19:27:00
"Sweden: GDP: Gross Fixed Investment (SA, Mil.SEK)","405,057",2025-06-30,"396,879","8,178",1.3,"3,982","11,005",2025-11-12 19:27:00
"Sweden: Change in Inventories including Valuables (SA, Mil.SEK)","8,065",2025-06-30,"5,215","2,850",0.37,"-8,273","17,707",2025-11-12 19:27:00
"Sweden: Final Consumption Expenditure: General Government (SA, Mil.SEK)","432,388",2025-06-30,"428,318","4,070",2.0,"6,825","10,507",2025-11-12 19:27:00
"Sweden: Final Consumption Expenditure: Households and NPISH (SA, Mil.SEK)","749,700",2025-06-30,"740,999","8,701",1.3,"25,747","29,950",2025-11-12 19:27:00
"Sweden: GDP: Exports of Goods and Services (SA, Mil.SEK)","868,912",2025-06-30,"887,722","-18,810",-1.3,-658.0,"-6,594",2025-11-12 19:27:00
"Sweden: GDP: Imports of Goods and Services (SA, Mil.SEK)","841,473",2025-06-30,"837,602","3,871",0.31,"10,237","19,904",2025-11-12 19:27:00


In [7]:
growth_stats_collection.dashboard.plots(show_0=True)

#### Nominal GDP

In [8]:
ngdp = hc.get_series(f"H{haver_code}NGDP@G10")
ngdp_ann = ngdp.ts.rolling(4).sum()
plot_line(
    ngdp_ann.pct_change(4),
    title="Nominal GDP Growth (%)",
    tick_format="0.0%",
    show_0=True,
)
# ngdp.pct_change(4).plot(show_0=True, years_limit=4) # Nominal GDP Growth

### Trade

In [9]:
df_td = hc.get_series("SETIB@NORDIC").ts.rename("Trade Balance (SEK bn)")
df_er = hc.get_series("SENXUSV@NORDIC").ts.rename("SEK/USD")

fig = plot_line(
    blue=df_td,
    red=df_er,
    title="<b>Sweden: Trade Balance vs SEK/USD Exchange Rate</b>",
    tick_suffix="",
    years_limit=10,
    watermark=False,
    source="Haver",
    # figsize=(1000, 600)
)

fig.data[1].update(yaxis="y2")

fig.update_layout(
    yaxis=dict(
        title=dict(text="Trade Balance (SEK bn)", font=dict(color="blue")),
        tickfont=dict(color="blue"),
    ),
    yaxis2=dict(
        title=dict(text="SEK/USD", font=dict(color="red")),
        tickfont=dict(color="red"),
        overlaying="y",
        side="right",
    ),
)

fig.show()

### PMI

In [10]:
fig = pmi.plot()
fig.add_hline(y=50, line_width=1, line_dash="dash", line_color="black")
fig.show()

### Confidence Indicator

In [11]:
fig = confidence_indicator.plot()
fig.add_hline(y=100, line_width=1, line_dash="dash", line_color="black")
fig.show()

### Riksbank Metrics

In [12]:
riksbank_sentiment = {
    "Business Price Expectations": "SESVBSPM@NORDIC",
    "Consumer Confidence": "SESCCI@NORDIC",
    "Consumer Confidence Macro": "SESCCIMA@NORDIC",
    "Consumer Confidence Micro": "SESCCIMI@NORDIC",
    "Business Demand Assessment": "SESVBODM@NORDIC",
    "1Y CPIF Expectations": "SENJ1FA@NORDIC",
    "2Y CPIF Expectations": "SENJ2FA@NORDIC",
    "5Y CPIF Expectations": "SENJ5FA@NORDIC",
}

riksbank_sentiment_collection = []
for k, v in riksbank_sentiment.items():
    riksbank_sentiment_collection.append(hc.get_series(v))

riksbank_sentiment_collection = TulipCollection(riksbank_sentiment_collection)
riksbank_sentiment_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Sweden: Business Tendency Survey: Selling Price Expectations (SA, %)",14.0,2025-10-31,13.0,1.0,0.30,-7.0,3.0,2025-11-12 19:27:00
"Sweden: Consumer Confidence Indicator (SA, 100=Mean)",96.8,2025-10-31,93.3,3.5,1.3,14.6,-4.0,2025-11-12 19:27:00
"Sweden: Consumer Confidence Indicator: Macro Index (SA, 100=Mean)",100.3,2025-10-31,97.4,2.9,1.1,11.8,-6.4,2025-11-12 19:27:00
"Sweden: Consumer Confidence Indicator: Micro Index (SA, 100=Mean)",93.9,2025-10-31,90.7,3.2,1.1,14.5,-0.40,2025-11-12 19:27:00
"Sweden: Business Tendency Survey: Demand, Assessment (SA, %)",-27.0,2025-10-31,-28.0,1.0,0.28,2.0,9.0,2025-11-12 19:27:00
"Sweden: Exp Yr1 CPIF: Money Market Players: Mean (NSA, %)",1.6,2025-10-31,1.9,-0.37,-1.9,-0.60,-0.13,2025-11-12 19:27:00
"Sweden: Exp Yr2 CPIF: Money Market Players: Mean (NSA, %)",1.9,2025-10-31,2.0,-0.09,-0.76,-0.35,0.10,2025-11-12 19:27:00
"Sweden: Exp Yr5 CPIF: Money Market Players: Mean (NSA, %)",2.2,2025-10-31,2.1,0.05,0.64,-0.12,0.13,2025-11-12 19:27:00


### Inflation

In [13]:
hc.get_series("SENPCY@NORDIC").plot()

### Unemployment

In [14]:
unemployment_rate = hc.get_series("SESELUR@NORDIC")
plot_line(
    unemployment_rate.ts,
    title="Sweden: Labor Force Survey Unemployment Rate (SA, %)",
    tick_suffix="%",
)

Sweeden publishes a weekly figure on unemployment. We try to take advantage of it by normalizing by the labor force. The result differs from the monthly figure but we chart it for the purpose of having a faster measure.

In [15]:
unemployed = hc.get_series("E144TVAR@INTWKLY").ts.rename("All Registered Unemployed")
labor_force = hc.get_series("SESELLF@NORDIC ").ts.rename("Labor Force")
labor_force = labor_force.mul(100).resample("W-MON").bfill()
weekly_unemployment = (
    pd.concat(
        [
            unemployed,
            labor_force,
            unemployment_rate.ts.div(100)
            .resample("W-MON")
            .bfill()
            .rename("Labor Force Survey Unemployment"),
        ],
        axis=1,
    )
    .ffill()
    .dropna()
)
weekly_unemployment["Fast Unemployment"] = (
    weekly_unemployment["All Registered Unemployed"]
    / weekly_unemployment["Labor Force"]
)

fig = plot_lines(
    weekly_unemployment[["Labor Force Survey Unemployment", "Fast Unemployment"]],
    title="<b>SPES-Based Unemployment Rate (Weekly figure)</b>",
    axis_title="Unemployment Rate (%)",
    years_limit=2,
    # figsize=(1000, 600),
    source="Haver",
    watermark=False,
    tick_format="0.1%",
)

fig.show()

In [16]:
urate = hc.get_urate("SWE")
nairu_cb = hc.get_series(
    "SEAVELUR@NORDIC"
)  #    [Sweden:  NIER Equilibrium Unemployment, Percent of Potential Labor Force (%)]
plot_lines(
    [urate.ts.rename("Unemployment"), nairu_cb.ts.rename("Nairu Estimate")],
    default_x_range=("2010-01-01", "2025-12-31"),
    title="Swedish Unemployment vs Nairu",
    tick_suffix="%",
)

### Fiscal Stance

In [17]:
fiscal = {
    "Budget Deficit": "SENFGB@NORDIC",
    # "Budget Deficit annual": "SEAFGGB@NORDIC",
    "General Govt Budget Deficit": "SEAFTNL@NORDIC",
    "Defense Expenditure": "SENFEND@NORDIC",
    "Central Govt Debt": "SENFPD@NORDIC",
}
fiscal_collection = hc.create_collection(list(fiscal.values()))

In [18]:
deficit_as_PGDP = (
    fiscal_collection[0].ts.rolling(12).sum() / ngdp_ann.resample("ME").ffill()
)
plot_lines(
    deficit_as_PGDP.rename("Deficit as percentage of GDP"),
    tick_format="0.0%",
    title="<b>Sweeden Deficit as percentage of GDP</b>",
    years_limit=3,
    show_0=True,
)

### Credit Creation
#### Outstanding Levels

In [19]:
df_dd = hc.get_series("SESZLDDG@NORDIC").ts.rename("Domestic Debt")
df_hd = hc.get_series("SESZDHP@NORDIC").ts.rename("Household Debt")
df_nfd = hc.get_series("SESZDNP@NORDIC").ts.rename("Nonfinancial Corporations Debt")
df_fd = hc.get_series("SESZDFP@NORDIC").ts.rename("Financial Corporations Debt")
df_gd = hc.get_series("SESZDGP@NORDIC").ts.rename("Government Debt")
df_nd = hc.get_series("SESZLSP@NORDIC").ts.rename(
    "Nonprofit Institutions Serving Households' Debt"
)

combined_df = pd.concat([df_dd, df_hd, df_nfd, df_fd, df_gd, df_nd], axis=1).dropna()

fig = plot_lines(
    combined_df,
    title="",
    tick_suffix="%",
    y_axis_label="Debt Outstanding as a % of SA GDP</b>",
    x_axis_label="Date",
    watermark=False,
    years_limit=10,
    source="Haver",
    logo=False,
    # figsize=(1000, 600)
)

fig.show()

In [20]:
yoy_df = combined_df.diff(6) * 2
yoy_df = yoy_df.dropna()

fig = plot_lines(
    yoy_df,
    title="",
    tick_suffix="%",
    y_axis_label="Debt Creation (Ann. Chg in % of GDP)",
    x_axis_label="Date",
    watermark=False,
    years_limit=10,
    source="Haver",
    logo=False,
    # figsize=(1000, 600)
)

fig.show()

#### Housing Consumption and Debt

In [21]:
df_od = hc.get_series("SESZLHDI@NORDIC").ts.rename(
    "Outstanding Debt to Disposable Income"
)
df_ir = hc.get_series("SENRREPV@NORDIC").ts.rename("Interest Rate")

merged_df = pd.concat([df_od, df_ir], axis=1).dropna()

fig = plot_line(
    blue=merged_df["Outstanding Debt to Disposable Income"],
    red=merged_df["Interest Rate"],
    title="<b>% of Outstanding Debt to Disposable Income vs Interest Rate</b>",
    tick_suffix="%",
    y_axis_label="Debt to Disposable Income (%)",
    y2_axis_label="Interest Rate (%)",
    source="Haver",
    watermark=False,
    years_limit=10,
    # figsize=(1000, 600)
)

fig.show()

### Foreign Exchange Rate

In [22]:
hc.get_series("SENXUSV@NORDIC").plot()

### Assets 
#### Real Estate

In [23]:
df_policy = hc.get_series("SENRREPV@NORDIC").ts.rename("Policy Rate")
df_2m = hc.get_series("SENRM2@NORDIC").ts.rename("2Y Mortgage Yield")
df_5m = hc.get_series("SENRM5@NORDIC").ts.rename("5Y Mortgage Yield")
df_hp = hc.get_series("SEAPH001@NORDIC").ts.rename("Housing Prices")

df_all = pd.concat([df_2m, df_5m, df_policy, df_hp], axis=1).dropna()

fig = plot_lines(
    df_all,
    title="<b>Swedish Mortgage Yields, Policy Rate, and Housing Prices</b>",
    width=1000,
    height=600,
    tick_suffix="%",
    watermark=False,
    source="Haver",
    default_y_range=(-10, 15),
)

switch_trace_to_secondary_axis(
    fig,
    trace_names="Housing Prices",
    secondary_axis_title="Housing Price Index (1981 = 100)",
    tick_suffix="",  # No % on right
)

fig.update_layout(yaxis=dict(title="Mortgage Yields & Policy Rate (%)"))

fig.show()

In [24]:
omx = hc.get_series("S144O30@INTWKLY").ts.rename("OMXS30")
spx = hc.get_series("S111SP5@INTWKLY").ts.rename("S&P500")

df = pd.concat([omx, spx], axis=1).dropna()
df = df[df.index >= "2020-01-01"]

df_rebased = df / df.iloc[0] * 100

fig = plot_lines(
    df_rebased,
    title="<b>Rebased Index Comparison: OMXS30 vs. S&P 500 (Start = 100)</b>",
    axis_title="Index Level (Rebased)",
    source="Haver",
    watermark=False,
    width=1000,
    height=600,
)

fig.show()

In [25]:
# fig = pmi.plot()
# fig.add_hline(y=50, line_width=1, line_dash="dash", line_color="black")
# fig.show()


In [26]:
# fig = confidence_indicator.plot()
# fig.add_hline(y=100, line_width=1, line_dash="dash", line_color="black")
# fig.show()

In [27]:
# urate = hc.get_urate("SWE")
# nairu_cb = hc.get_series("SEAVELUR@NORDIC") #    [Sweden:  NIER Equilibrium Unemployment, Percent of Potential Labor Force (%)]
# plot_lines([urate.ts.rename('Unemployment'), nairu_cb.ts.rename('Nairu Estimate')], default_x_range=('2010-01-01', '2025-12-31'), title="Swedish Unemployment vs Nairu", tick_suffix = '%')

### Riksbank Indicator (Leads Interest Rate Cuts)

In [28]:
riksbank_sentiment = {
    "Business Price Expectations": "SESVBSPM@NORDIC",
    "Consumer Confidence": "SESCCI@NORDIC",
    "Consumer Confidence Macro": "SESCCIMA@NORDIC",
    "Consumer Confidence Micro": "SESCCIMI@NORDIC",
    "Business Demand Assessment": "SESVBODM@NORDIC",
    "1Y CPIF Expectations": "SENJ1FA@NORDIC",
    "2Y CPIF Expectations": "SENJ2FA@NORDIC",
    "5Y CPIF Expectations": "SENJ5FA@NORDIC",
}

riksbank_sentiment_collection = []
for k, v in riksbank_sentiment.items():
    riksbank_sentiment_collection.append(hc.get_series(v))

riksbank_sentiment_collection = TulipCollection(riksbank_sentiment_collection)
riksbank_sentiment_collection.dashboard.table()

,Last Value,Last Date,Previous Value,Change Since Previous,Change Since Previous Z,Change 6M,Change 12M,Updated
"Sweden: Business Tendency Survey: Selling Price Expectations (SA, %)",14.0,2025-10-31,13.0,1.0,0.30,-7.0,3.0,2025-11-12 19:28:00
"Sweden: Consumer Confidence Indicator (SA, 100=Mean)",96.8,2025-10-31,93.3,3.5,1.3,14.6,-4.0,2025-11-12 19:28:00
"Sweden: Consumer Confidence Indicator: Macro Index (SA, 100=Mean)",100.3,2025-10-31,97.4,2.9,1.1,11.8,-6.4,2025-11-12 19:28:00
"Sweden: Consumer Confidence Indicator: Micro Index (SA, 100=Mean)",93.9,2025-10-31,90.7,3.2,1.1,14.5,-0.40,2025-11-12 19:28:00
"Sweden: Business Tendency Survey: Demand, Assessment (SA, %)",-27.0,2025-10-31,-28.0,1.0,0.28,2.0,9.0,2025-11-12 19:28:00
"Sweden: Exp Yr1 CPIF: Money Market Players: Mean (NSA, %)",1.6,2025-10-31,1.9,-0.37,-1.9,-0.60,-0.13,2025-11-12 19:28:00
"Sweden: Exp Yr2 CPIF: Money Market Players: Mean (NSA, %)",1.9,2025-10-31,2.0,-0.09,-0.76,-0.35,0.10,2025-11-12 19:28:00
"Sweden: Exp Yr5 CPIF: Money Market Players: Mean (NSA, %)",2.2,2025-10-31,2.1,0.05,0.64,-0.12,0.13,2025-11-12 19:28:00


In [29]:
riksbank_sentiment_collection.dashboard.plots()

Riksbank Sentiment Index [see](https://markets.jpmorgan.com/jpmm/research.article_page?action=open&doc=GPS-4990016-0)

In [30]:
sentiment_indicator = pd.DataFrame(
    {
        "industry_demands": riksbank_sentiment_collection["SESVBODM@NORDIC"].ts,
        "industry_price_expectations": riksbank_sentiment_collection[
            "SESVBSPM@NORDIC"
        ].ts,
        "confidence_indicator": riksbank_sentiment_collection["SESCCI@NORDIC"].ts,
        "cpi_expectations": riksbank_sentiment_collection["SENJ5FA@NORDIC"].ts,
    }
)
ssi = (
    sentiment_indicator.sub(sentiment_indicator.mean())
    .div(sentiment_indicator.std())
    .mean(axis=1)
)

policy_rate_target = bb.get_series("SWRRATEI Index")
ssi = ssi.rename("Riksbank Sentiment Index").to_frame()
ssi["Policy Rate Target"] = policy_rate_target.ts.resample("ME").last().ffill()
ssi["Policy Rate Change"] = ssi["Policy Rate Target"].diff()
fig = plot_line(
    blue=ssi.loc["2003":, "Policy Rate Change"],
    red=ssi.loc["2003":, "Riksbank Sentiment Index"],
    show_0=True,
    align_0=True,
)
fig.show()

Source: J.P. Morgan, SCB, Riksbank, NIER, Origo. *Z scores of NIER industry demand exp., 
NIER industry price exp., HH conf., Origo 5-year CPI exp. Four variables have equal weights

In [31]:
Markdown(f"_Notebook updated at {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M')}_")

_Notebook updated at 2025-11-12 19:28_